# Reconhecimento de entidades nomeadas

Uma grande empresa de advocacia está buscando soluções tecnológicas para otimizar o processamento de documentos jurídicos e melhorar a gestão dos processos em andamento.

Atualmente, o escritório lida com um volume significativo de contratos, petições, jurisprudências e outros documentos legais, mas não há uma forma eficiente de extrair automaticamente informações críticas desses textos, como nomes de partes envolvidas, datas relevantes e dispositivos legais citados.

Nosso papel nesse projeto é desenvolver um modelo de IA especializado em Reconhecimento de Entidades Nomeadas (NER), capaz de identificar e classificar automaticamente essas informações em textos jurídicos. Isso permitirá que os advogados e equipes jurídicas tenham acesso mais rápido a dados importantes, otimizando o tempo de análise e reduzindo o risco de erro humano, além de facilitar a gestão dos processos de forma mais eficiente.

## Leitura dos textos

Vamos utilizar dados de arquivos txt que estão compactados em um arquivo zip. Para abrir esses arquivos, vamos usar a biblioteca `zipfile`.

In [1]:
import zipfile

In [2]:
with zipfile.ZipFile('/content/texts.zip', 'r') as zip:
    print(zip.namelist())

['ADI2TJDFT.txt', 'adi3767.txt', 'Ag10000170733596001.txt', 'Ag10105170208398001.txt', 'AgAIRR11889820145030011.txt', 'AgCr10582160008758001.txt', 'AgRgSTJ1.txt', 'AgRgSTJ2.txt', 'AgRgTSE1.txt', 'AgRgTSE3.txt', 'AIAgRAgI6193ARAGUARIMG.txt', 'airr801422012.txt', 'AIRR3999520145020086.txt', 'AIRR15708820115050222.txt', 'AP00000794920137060006.txt', 'AP00001415620157010201.txt', 'AP00001441420167030203.txt', 'AP771420167080008PA.txt', 'CP32320177080008PA.txt', 'DespSEPLAGDF.txt', 'ED1STM.txt', 'ED1TJAC.txt', 'EDAgRgTSE2.txt', 'EDEDARR208420135040232.txt', 'EDRR1TST.txt', 'EEDRR9715120105020002.txt', 'ERR731004520105130003.txt', 'HC110260SP.txt', 'HC151914AgRES.txt', 'HC340624SP.txt', 'HC418951PR.txt', 'HC70000845920187000000.txt', 'lei11340.txt', 'Lei11788.txt', 'LoaDF2018.txt', 'Pet128TSE5.txt', 'Port77DF.txt', 'Rcl3495STJ.txt', 'REE5908TSE4.txt', 'REsp1583083RS.txt', 'RR474820145230056.txt', 'RR942006420095040028.txt', 'RR2574407120025020372.txt', 'TCU4687.txt', 'TSTRR16037920105200001.

In [3]:
with zipfile.ZipFile('/content/texts.zip', 'r') as zip:
    print(*zip.namelist(), sep = '\n')

ADI2TJDFT.txt
adi3767.txt
Ag10000170733596001.txt
Ag10105170208398001.txt
AgAIRR11889820145030011.txt
AgCr10582160008758001.txt
AgRgSTJ1.txt
AgRgSTJ2.txt
AgRgTSE1.txt
AgRgTSE3.txt
AIAgRAgI6193ARAGUARIMG.txt
airr801422012.txt
AIRR3999520145020086.txt
AIRR15708820115050222.txt
AP00000794920137060006.txt
AP00001415620157010201.txt
AP00001441420167030203.txt
AP771420167080008PA.txt
CP32320177080008PA.txt
DespSEPLAGDF.txt
ED1STM.txt
ED1TJAC.txt
EDAgRgTSE2.txt
EDEDARR208420135040232.txt
EDRR1TST.txt
EEDRR9715120105020002.txt
ERR731004520105130003.txt
HC110260SP.txt
HC151914AgRES.txt
HC340624SP.txt
HC418951PR.txt
HC70000845920187000000.txt
lei11340.txt
Lei11788.txt
LoaDF2018.txt
Pet128TSE5.txt
Port77DF.txt
Rcl3495STJ.txt
REE5908TSE4.txt
REsp1583083RS.txt
RR474820145230056.txt
RR942006420095040028.txt
RR2574407120025020372.txt
TCU4687.txt
TSTRR16037920105200001.txt
AC1TCU.txt
AC1TJAC.txt
AC1TJMG.txt
AC2.txt
ACORDAOTCU25052016.txt


In [4]:
with zipfile.ZipFile('/content/texts.zip', 'r') as zip:
    with zip.open('ADI2TJDFT.txt') as arquivo:
        texto = arquivo.read().decode('utf-8')

In [5]:
print(texto)

Órgão	:	Conselho Especial
Classe	:	ADI  Ação Direta de Inconstitucionalidade
N. Processo	:	2010002019357-4
Requerente(s)	:	PROCURADORA-GERAL DE JUSTIÇA DO DISTRITO FEDERAL E TERRITÓRIOS
Requerido(s)	:	PRESIDENTE DA CÂMARA LEGISLATIVA DO DISTRITO FEDERAL E OUTRO(S)
Relator 	:	Desembargador LÉCIO RESENDE
	EMENTA	
AÇÃO DIRETA DE INCONSTITUCIONALIDADE. LEIS DISTRITAIS N.º 747/1994 E 2018/1998. LEI COMPLEMENTAR DISTRITAL N.º 380/2001. INCONSTITUCIONALIDADE FORMAL. LEI ORGÂNICA DO DISTRITO FEDERAL. OCUPAÇÃO DE ÁREA PÚBLICA. COMPETÊNCIA PRIVATIVA DO GOVERNADOR DO DISTRITO FEDERAL. AÇÃO JULGADA PROCEDENTE EM RAZÃO DO VÍCIO FORMAL. Tanto o Decreto n.º 10.829/87, quanto a Portaria n.º 314/92, do Instituto Brasileiro do Patrimônio Cultural  IBPC, hoje Instituto do Patrimônio Histórico e Artístico Nacional  IPHAN, conferem ao Governador do Distrito Federal competência privativa para iniciar o processo legislativo, quando se tratar o tema de uso e ocupação do solo em todo o território do Distrito F

## Reconhecendo as entidades no texto

Para fazer o reconhecimento das entidades nos textos, vamos usar modelos da biblioteca `spacy`. Podemos utilizar modelos já treinados e personalizar esses modelos para atender ao nosso problema específico.

A documentação fornece detalhes de como fazer a instalação do modelo com base no sistema operacional e a língua que desejarmos.

https://spacy.io/usage

In [6]:
# !pip install -U spacy

In [7]:
# !python -m spacy download pt_core_news_sm

In [8]:
import spacy
import pt_core_news_sm

modelo_ner = pt_core_news_sm.load()

In [9]:
doc = modelo_ner(texto)

for entidade in doc.ents:
    print(f'{entidade.text} -> {entidade.label_}')

Órgão -> LOC
Conselho Especial
Classe -> ORG
DISTRITO FEDERAL E TERRITÓRIOS
Requerido(s -> LOC
DA CÂMARA -> ORG
DISTRITO FEDERAL E OUTRO(S)
Relator -> LOC
LEIS DISTRITAIS -> MISC
COMPLEMENTAR DISTRITAL -> MISC
FORMAL -> ORG
LEI ORGÂNICA -> MISC
DISTRITO FEDERAL -> LOC
ÁREA PÚBLICA -> LOC
DO GOVERNADOR -> MISC
DISTRITO FEDERAL -> LOC
VÍCIO FORMAL -> ORG
Decreto n.º 10.829/87 -> MISC
Portaria n.º 314/92 -> ORG
Instituto Brasileiro do Patrimônio Cultural  IBPC -> LOC
Instituto do Patrimônio Histórico e Artístico Nacional  IPHAN -> LOC
Governador do Distrito Federal -> LOC
Distrito Federal -> LOC
ACÓRDÃO -> ORG
Acordam os Desembargadores do Conselho Especial do Tribunal de Justiça do Distrito Federal -> MISC
Territórios LÉCIO RESENDE  Relator -> ORG
JOÃO MAIORISI -> PER
Vogal -> ORG
VIEIRA  Vogal -> LOC
SÉRGIO BITTENCOURT  Vogal -> ORG
LECIR MANOEL DA LUZ -> ORG
Vogal -> LOC
CARMELITA BRASIL -> ORG
Vogal -> LOC
WALDIR LEÔNCIO -> PER
J.J. COSTA CARVALHO -> PER
Vogal -> LOC
HUMBERTO -> LOC
V

In [10]:
import pandas as pd

In [11]:
entidades = []
labels = []

for entidade in doc.ents:
    entidades.append(entidade.text)
    labels.append(entidade.label_)

In [12]:
pd.DataFrame({'Entidade': entidades, 'Label': labels})

,Entidade,Label
0,Órgão,LOC
1,Conselho Especial\nClasse,ORG
2,DISTRITO FEDERAL E TERRITÓRIOS\nRequerido(s,LOC
3,DA CÂMARA,ORG
4,DISTRITO FEDERAL E OUTRO(S)\nRelator,LOC
...,...,...
851,PROCEDENTE,LOC
852,Leis Distritais,MISC
853,Lei Complementar,MISC
854,omnes,ORG


## Visualizando as entidades no texto

In [16]:
modelo_ner.get_pipe('ner').labels

('LOC', 'MISC', 'ORG', 'PER')

In [17]:
print('LOC', spacy.explain('LOC'))
print('MISC', spacy.explain('MISC'))
print('ORG', spacy.explain('ORG'))
print('PER', spacy.explain('PER'))

LOC Non-GPE locations, mountain ranges, bodies of water
MISC Miscellaneous entities, e.g. events, nationalities, products or works of art
ORG Companies, agencies, institutions, etc.
PER Named person or family.


In [18]:
spacy.displacy.render(doc, style = 'ent', jupyter=True)

In [19]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 59.2 MB/s eta 0:00:00
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 3.7.1
    Uninstalling en-core-web-sm-3.7.1:
      Successfully uninstalled en-core-web-sm-3.7.1
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [20]:
import en_core_web_sm

modelo_ner_ing = spacy.load('en_core_web_sm')
modelo_ner_ing.get_pipe('ner').labels

('CARDINAL',
 'DATE',
 'EVENT',
 'FAC',
 'GPE',
 'LANGUAGE',
 'LAW',
 'LOC',
 'MONEY',
 'NORP',
 'ORDINAL',
 'ORG',
 'PERCENT',
 'PERSON',
 'PRODUCT',
 'QUANTITY',
 'TIME',
 'WORK_OF_ART')

## Tokenizando os dados

In [21]:
dados = []

with zipfile.ZipFile('/content/texts.zip') as zip:
    for nome_arquivo in zip.namelist():
        with zip.open(nome_arquivo) as arquivo:
            conteudo = arquivo.read().decode('utf-8')
            palavras = conteudo.split()
            for palavra in palavras:
                dados.append([nome_arquivo, palavra])

tabela_palavras = pd.DataFrame(dados, columns = ['arquivo', 'palavra'])

In [22]:
tabela_palavras

,arquivo,palavra
0,ADI2TJDFT.txt,Órgão
1,ADI2TJDFT.txt,:
2,ADI2TJDFT.txt,Conselho
3,ADI2TJDFT.txt,Especial
4,ADI2TJDFT.txt,Classe
...,...,...
193487,ACORDAOTCU25052016.txt,Ministro-Substituto
193488,ACORDAOTCU25052016.txt,AUGUSTO
193489,ACORDAOTCU25052016.txt,SHERMAN
193490,ACORDAOTCU25052016.txt,CAVALCANTI


In [23]:
tabela_palavras.to_csv('palavras.csv', index = False, sep = '\t')

## Rotulando os dados no formato IOB

O formato BIO/IOB ,abreviação de inside, outside, beginning, é um formato de marcação comum para marcar tokens em uma tarefa de agrupamento em linguística computacional (por exemplo, reconhecimento de entidade nomeada).

- O prefixo B antes de uma tag indica que a tag é o início de um pedaço
- O prefixo I antes de uma tag indica que a tag está dentro de um pedaço.
- O prefixo O indica que um token não pertence a nenhuma entidade/pedaço.

Exemplo:


| **palavra** |     **label** |
|------------:|--------------:|
|     ACORDAO |             O |
|    CONSELHO | B-ORGANIZACAO |
|    ESPECIAL | I-ORGANIZACAO |
|          DF |       B-LOCAL |


In [24]:
tabela_palavras = pd.read_csv('/content/palavras_IOB.tsv', sep = '\t')

In [25]:
tabela_palavras

,arquivo,palavra,label
0,TCU4687.txt,GRUPO,O
1,TCU4687.txt,I,O
2,TCU4687.txt,CLASSE,O
3,TCU4687.txt,II,O
4,TCU4687.txt,2ª,B-ORGANIZACAO
...,...,...,...
229272,RR2574407120025020372.txt,de,O
229273,RR2574407120025020372.txt,Chaves,O
229274,RR2574407120025020372.txt,Públicas,O
229275,RR2574407120025020372.txt,Brasileira,O


In [26]:
tabela_palavras['label'].unique()

array(['O', 'B-ORGANIZACAO', 'I-ORGANIZACAO', 'B-JURISPRUDENCIA',
       'I-JURISPRUDENCIA', 'B-PESSOA', 'I-PESSOA', 'B-LEGISLACAO',
       'I-LEGISLACAO', 'B-TEMPO', 'B-LOCAL', 'I-LOCAL', 'I-TEMPO'],
      dtype=object)

## Transformando um arquivo para o formato spacy

Formato Spacy:

```
tupla = ('texto', {'entities': [(10, 17, 'ORGANIZACAO'), (25,40, 'PESSOA')]})
```

In [28]:
grupos_arquivos = tabela_palavras.groupby(by= 'arquivo')

In [29]:
grupos_arquivos.get_group('ADI2TJDFT.txt')

,arquivo,palavra,label
46105,ADI2TJDFT.txt,Órgão,O
46106,ADI2TJDFT.txt,:,O
46107,ADI2TJDFT.txt,Conselho,B-ORGANIZACAO
46108,ADI2TJDFT.txt,Especial,I-ORGANIZACAO
46109,ADI2TJDFT.txt,Classe,O
...,...,...,...
55911,ADI2TJDFT.txt,a,O
55912,ADI2TJDFT.txt,ação,O
55913,ADI2TJDFT.txt,",",O
55914,ADI2TJDFT.txt,maioria,O


In [32]:
tabela_agrupada = grupos_arquivos.get_group('ADI2TJDFT.txt')[['palavra', 'label']].values
conteudo = ''
anotacoes = {'entities': []}
inicio_palavra = 0
fim_palavra = 0
for texto, label in tabela_agrupada:
    texto = str(texto)
    tamanho_texto = len(texto)+1

    inicio_palavra = fim_palavra
    fim_palavra = inicio_palavra + tamanho_texto

    if label != 'O':
        anotacao = (inicio_palavra, fim_palavra-1, label)
        anotacoes['entities'].append(anotacao)

    conteudo = conteudo + texto + ' '

In [33]:
conteudo

"Órgão : Conselho Especial Classe : ADI Ação Direta de Inconstitucionalidade N . Processo : 2010002019357-4 Requerente ( s ) : PROCURADORA-GERAL DE JUSTIÇA DO DISTRITO FEDERAL E TERRITÓRIOS Requerido ( s ) : PRESIDENTE DA CÂMARA LEGISLATIVA DO DISTRITO FEDERAL E OUTRO ( S ) Relator : Desembargador LÉCIO RESENDE EMENTA AÇÃO DIRETA DE INCONSTITUCIONALIDADE . LEIS DISTRITAIS N.º 747/1994 E 2018/1998 . LEI COMPLEMENTAR DISTRITAL N.º 380/2001 . INCONSTITUCIONALIDADE FORMAL . LEI ORGÂNICA DO DISTRITO FEDERAL . OCUPAÇÃO DE ÁREA PÚBLICA . COMPETÊNCIA PRIVATIVA DO GOVERNADOR DO DISTRITO FEDERAL . AÇÃO JULGADA PROCEDENTE EM RAZÃO DO VÍCIO FORMAL . Tanto o Decreto n.º 10.829/87 , quanto a Portaria n.º 314/92 , do Instituto Brasileiro do Patrimônio Cultural IBPC , hoje Instituto do Patrimônio Histórico e Artístico Nacional IPHAN , conferem ao Governador do Distrito Federal competência privativa para iniciar o processo legislativo , quando se tratar o tema de uso e ocupação do solo em todo o territ

In [34]:
anotacoes

{'entities': [(8, 16, 'B-ORGANIZACAO'),
  (17, 25, 'I-ORGANIZACAO'),
  (91, 106, 'B-JURISPRUDENCIA'),
  (158, 166, 'B-LOCAL'),
  (167, 174, 'I-LOCAL'),
  (221, 227, 'B-ORGANIZACAO'),
  (228, 239, 'I-ORGANIZACAO'),
  (240, 242, 'I-ORGANIZACAO'),
  (243, 251, 'I-ORGANIZACAO'),
  (252, 259, 'I-ORGANIZACAO'),
  (298, 303, 'B-PESSOA'),
  (304, 311, 'I-PESSOA'),
  (358, 362, 'B-LEGISLACAO'),
  (363, 373, 'I-LEGISLACAO'),
  (374, 377, 'I-LEGISLACAO'),
  (378, 386, 'I-LEGISLACAO'),
  (389, 398, 'B-LEGISLACAO'),
  (401, 404, 'B-LEGISLACAO'),
  (405, 417, 'I-LEGISLACAO'),
  (418, 427, 'I-LEGISLACAO'),
  (428, 431, 'I-LEGISLACAO'),
  (432, 440, 'I-LEGISLACAO'),
  (474, 477, 'B-LEGISLACAO'),
  (478, 486, 'I-LEGISLACAO'),
  (487, 489, 'I-LEGISLACAO'),
  (490, 498, 'I-LEGISLACAO'),
  (499, 506, 'I-LEGISLACAO'),
  (575, 583, 'B-LOCAL'),
  (584, 591, 'I-LOCAL'),
  (653, 660, 'B-LEGISLACAO'),
  (661, 664, 'I-LEGISLACAO'),
  (665, 674, 'I-LEGISLACAO'),
  (686, 694, 'B-LEGISLACAO'),
  (695, 698, 'I-LEGIS

In [35]:
conteudo.find('Conselho Especial')

8

In [36]:
conteudo.find('Conselho Especial') + len('Conselho Especial')

25

## Transformando todos os dados

In [38]:
arquivos = grupos_arquivos.groups.keys()

In [39]:
arquivos

dict_keys(['AC1TCU.txt', 'AC1TJAC.txt', 'AC1TJMG.txt', 'AC2.txt', 'ACORDAOTCU25052016.txt', 'ADI2TJDFT.txt', 'AIAgRAgI6193ARAGUARIMG.txt', 'AIRR15708820115050222.txt', 'AIRR3999520145020086.txt', 'AP00000794920137060006.txt', 'AP00001415620157010201.txt', 'AP00001441420167030203.txt', 'AP771420167080008PA.txt', 'Ag10000170733596001.txt', 'Ag10105170208398001.txt', 'AgAIRR11889820145030011.txt', 'AgCr10582160008758001.txt', 'AgRgSTJ1.txt', 'AgRgSTJ2.txt', 'AgRgTSE1.txt', 'AgRgTSE3.txt', 'CP32320177080008PA.txt', 'DespSEPLAGDF.txt', 'ED1STM.txt', 'ED1TJAC.txt', 'EDAgRgTSE2.txt', 'EDEDARR208420135040232.txt', 'EDRR1TST.txt', 'EEDRR9715120105020002.txt', 'ERR731004520105130003.txt', 'HC110260SP.txt', 'HC151914AgRES.txt', 'HC340624SP.txt', 'HC418951PR.txt', 'HC70000845920187000000.txt', 'Lei11788.txt', 'LoaDF2018.txt', 'Pet128TSE5.txt', 'Port77DF.txt', 'REE5908TSE4.txt', 'REsp1583083RS.txt', 'RR2574407120025020372.txt', 'RR474820145230056.txt', 'RR942006420095040028.txt', 'Rcl3495STJ.txt', 

In [40]:
documentos = []
for arquivo in arquivos:
    documento = []
    tabela_agrupada = grupos_arquivos.get_group(arquivo)[['palavra', 'label']].values
    conteudo = ''
    anotacoes = {'entities': []}
    inicio_palavra = 0
    fim_palavra = 0
    for texto, label in tabela_agrupada:
        texto = str(texto)
        tamanho_texto = len(texto)+1

        inicio_palavra = fim_palavra
        fim_palavra = inicio_palavra + tamanho_texto

        if label != 'O':
            anotacao = (inicio_palavra, fim_palavra-1, label)
            anotacoes['entities'].append(anotacao)

        conteudo = conteudo + texto + ' '

    documento = (conteudo, anotacoes)
    documentos.append(documento)

In [41]:
documentos

[('Número do Acórdão ACÓRDÃO 2924/2017 - PLENÁRIO Relator BRUNO DANTAS Processo 021.074/2016-0 Tipo de processo DENÚNCIA ( DEN ) Data da sessão 12/12/2017 Número da ata 26/2017 Interessado / Responsável / Recorrente 3 . Denunciante/Responsáveis/Interessados : 3.1 . Denunciante : Identidade preservada ( art . 55 , caput , da Lei 8.443/1992 ) . 3.2 . Responsável : André Luiz Moreira da Silva ( 074.166.407-09 ) . 3.3 . Interessado : CSTrans Serviços de Transportes Ltda. ( 13.265.187/0001-05 ) . Entidade Comando da Aeronáutica - Grupamento de Apoio de Brasília . Representante do Ministério Público não atuou . Unidade Técnica Secretaria de Controle Externo de Aquisições Logísticas ( Selog ) . Representante Legal 8.1 . Mauro Santos Silva ( 054.218.307-21 ) e outros , representando Comando da Aeronáutica - Grupamento de Apoio de Brasília . 8.2 . Ivana Ferreira Castro Lobo Barbosa ( 718.698.321-91 ) , representando CSTrans Serviços de Transportes Ltda . Assunto Denúncia sobre irregularidades e